# Exercício Geração Automática de Texto

Utilizando Redes Neurais Recorrentes(RNR) personalizei um modelo para a criação de tweets baseado em um datase no site kaggle com alguns tweets do Elon Musk, ou seja, dessa vez não utilizei para a predição mas sim para a geração.

In [ ]:
# Imports
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

filename = "/content/drive/MyDrive/Colab Notebooks/RNR/data_elonmusk.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
chars = sorted(list(set(raw_text)))

In [ ]:
chars

In [ ]:
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
char_to_int

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)


In [ ]:
# À medida que dividimos o livro em sequências, convertemos os caracteres em números inteiros usando nossa
# tabela de pesquisa que preparamos anteriormente.
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total de Padrões: ", n_patterns)

In [ ]:
dataX

In [ ]:
# Reshape de X para [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Normalização
X = X / float(n_vocab)

In [ ]:
# One-Hot Encoding da variável de saída
y = np_utils.to_categorical(dataY)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.35))
model.add(LSTM(256))
model.add(Dropout(0.35))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Define o checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

In [ ]:
%%time
# model.fit(X, y, epochs = 20, batch_size = 128, callbacks = callbacks_list)
model.fit(X, y, epochs = 3, batch_size = 256, callbacks = callbacks_list)

In [ ]:
# Carrega os melhores pesos da rede e compila o modelo
filename = "/content/drive/MyDrive/Colab Notebooks/RNR/weights-improvement-03-1.5102.hdf5"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# Obtém um random seed
start = numpy.random.randint(0, len(dataX)-1)

# Inicia a geração de texto de um ponto qualquer, definido pelo random seed "start"
pattern = dataX[start]
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Gerando caracteres
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nConcluído.")

" rbx impact
       velocity and center of mass 2016-
       of colliding object make a big 10-10
row1 "
161hotps://t.co/xlallyzmm              03:33:33               elonmusk
                             
Concluído.
